In [1]:
#Importing the Dependencies

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
#Data Collection & Pre-Processing

In [2]:
# loading the data from csv file to a pandas Dataframe
raw_mail_data = pd.read_csv('mail_data.csv')
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [3]:
# replace the null values with a null string
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [4]:
# printing the first 5 rows of the dataframe
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# checking the number of rows and columns in the dataframe
mail_data.shape

(5572, 2)

In [ ]:
#Label Encoding

In [6]:
# label spam mail as 0;  ham mail as 1;

mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

In [7]:
# separating the data as texts and label

X = mail_data['Message']

Y = mail_data['Category']

In [8]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [9]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


In [ ]:
#Splitting the data into training data & test data

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [11]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


In [ ]:
#Feature Extraction

In [12]:
# transform the text data to feature vectors that can be used as input to the Logistic regression

feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase='True')

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

# convert Y_train and Y_test values as integers

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [ ]:
#Training the Model

In [ ]:
#Naive Bayes Machine Learning Model

In [13]:
prediction = dict()
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_features, Y_train)

MultinomialNB()

In [14]:
# prediction on training data

prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

print('Accuracy on training data using Naive Bayes Machine Learning Model : ', accuracy_on_training_data)

Accuracy on training data using Naive Bayes Machine Learning Model :  0.9807045097599282


In [15]:
# prediction on test data

prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

print('Accuracy on test data using Naive Bayes Machine Learning Model : ', accuracy_on_test_data)

Accuracy on test data using Naive Bayes Machine Learning Model :  0.9730941704035875


In [16]:
#Handling Imbalaced data using Undersampling and Oversampling

In [17]:
#checking the class distribution in the training set:
print('Class distribution in training set:')
print(Y_train.value_counts())


Class distribution in training set:
1    3865
0     592
Name: Category, dtype: int64


In [ ]:
#As we can see, the training set is highly imbalanced, with only 592 spam cases out 4457 total cases.
#To handle this imbalance, we can use under-sampling techniques.

In [18]:
#Undersampling

from imblearn.under_sampling import RandomUnderSampler

# Create an under-sampler object
rus = RandomUnderSampler(random_state=42)

# Apply under-sampling to the training set
X_train_resampled, Y_train_resampled = rus.fit_resample(X_train_features, Y_train)

# Check the class distribution in the resampled data
print('Class distribution in resampled training set:')
print(pd.Series(Y_train_resampled).value_counts())


Class distribution in resampled training set:
0    592
1    592
Name: Category, dtype: int64


In [22]:
#Now checking accuracy of undersampled data

from sklearn.naive_bayes import MultinomialNB

# Train a MultinomialNB model on the undersampled dataset
mnb_rus = MultinomialNB()
mnb_rus.fit(X_train_resampled, Y_train_resampled)

# Test the model
accuracy_rus = mnb_rus.score(X_test_features, Y_test)
print('Accuracy with undersampling: ', accuracy_rus)

Accuracy with undersampling:  0.9354260089686098


In [ ]:
#Building a Predictive System

In [23]:
input_mail = ["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_mail)

# making prediction
prediction = model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Ham mail')

else:
  print('Spam mail')

[1]
Ham mail


In [24]:
input_mail = ["FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"]

# convert text to feature vectors
input_data_features = feature_extraction.transform(input_mail)

# making prediction
prediction = model.predict(input_data_features)
print(prediction)


if (prediction[0]==1):
  print('Ham mail')

else:
  print('Spam mail')

[0]
Spam mail
